In [1]:
import random
import numpy as np
import sys 
import csv
import time
from scipy import stats
sys.path.append('..')
import math
import pandas as pd
from prior import prior
from liklihood import liklihood
from posterior import posterior
import graph_image_generator
import utilities
import matplotlib.pyplot as plt
from matplotlib import rc
import pylab as pl

[0.05316901 0.05316901 0.07007042 0.05316901 0.05316901 0.05316901
 0.05316901 0.05880282 0.07570423 0.05035211 0.09260563 0.07570423
 0.09260563 0.05316901 0.05316901 0.05880282]
hello


In [2]:
finalGraphs = []

In [3]:
prior_distribution_rating = 10

In [4]:
p_G_2args = prior(['a'], ['c'])
p_G_2args_prior_distribution =  p_G_2args.getDistribution(prior_distribution_rating)

p_G_3args = prior(['a','b'], ['c'])
p_G_3args_prior_distribution =  p_G_3args.getDistribution(prior_distribution_rating)

p_G_3args_2 = prior(['a','d'], ['c'])
p_G_3args_2_prior_distribution =  p_G_3args_2.getDistribution(prior_distribution_rating)

p_G_4args = prior(['a','d','b'], ['c'])
p_G_4args_prior_distribution =  p_G_4args.getDistribution(prior_distribution_rating)

p_G_5args = prior(['a','d','b'], ['c','e'])
p_G_5args_prior_distribution =  p_G_5args.getDistribution(prior_distribution_rating)

In [5]:
all_p_g = [p_G_2args, p_G_3args, p_G_3args_2, p_G_4args, p_G_5args]
all_prior_distribs = [p_G_2args_prior_distribution, 
                     p_G_3args_prior_distribution,
                     p_G_3args_2_prior_distribution,
                     p_G_4args_prior_distribution,
                     p_G_5args_prior_distribution]
index_p_G_s = list(range(len(all_p_g)))

In [6]:
def check_attack_holds(pos_args, neg_args, always_true, graph):
    
    all_args = pos_args + neg_args
    appropriate_graph = []
    
    for t in always_true:
        
        if t[0][0] not in all_args or t[0][1] not in all_args:
            appropriate_graph.append(False)
            continue
            
        attacker = all_args.index(t[0][0])
        attacked = all_args.index(t[0][1])
        
        if len(t) == 1:
            if graph[attacker, attacked] == 1 and graph[attacked, attacker] == 0:
                appropriate_graph.append(True)
            else:
                appropriate_graph.append(False)
            continue
            
        if len(t) == 2:
            if graph[attacker, attacked] == 1 and graph[attacked, attacker] == 1:
                appropriate_graph.append(True)
            else:
                appropriate_graph.append(False)
            continue
            
    return all(appropriate_graph)

In [ ]:
for i in range(0,16):
    print(i)
    print(check_attack_holds(all_p_g[1].pos_args, all_p_g[1].neg_args, 
                       [[('a','c'),('c','a')],[('c', 'b'),('b','c')]], 
                       all_p_g[1].arg_matrices[i]))

In [7]:
def generateSyntheticObservations(target_no_observations, all_p_g, all_prior_distribs, degree_of_appearance, always_true):

    observations = []
    index_p_G_s = list(range(len(all_p_g)))

    while len(observations) < target_no_observations:
        choice = random.choice(index_p_G_s) # choose an index from the list, index corresponds to one of the graph spaces
        chosen_prior = all_prior_distribs[choice] # store the prior for the graph space


        possible_graphs = []
        possible_graphs_probabilities = []

        appear_draw = random.uniform(0, 1)
        if appear_draw <= degree_of_appearance:
            for graph_no, graph in enumerate((all_p_g[choice]).arg_matrices):
                result = check_attack_holds(all_p_g[choice].pos_args, all_p_g[choice].neg_args, always_true, graph)
                if result:            
                    possible_graphs.append(graph)
                    possible_graphs_probabilities.append(chosen_prior[graph_no])

            if len(possible_graphs) ==0:
                continue
            possible_graphs_probabilities = possible_graphs_probabilities / np.sum(possible_graphs_probabilities)

        else:
            possible_graphs = (all_p_g[choice]).arg_matrices
            possible_graphs_probabilities = chosen_prior


        indices = list(range(len(possible_graphs_probabilities)))
        custm = stats.rv_discrete(name='custm', values=(indices, possible_graphs_probabilities))
        chosen_graph = custm.rvs(size=1)[0]

        observation = {}
        observation['pos_args'] = all_p_g[choice].pos_args
        observation['neg_args'] = all_p_g[choice].neg_args
        observation['attacks'] = utilities.convertArgMtrxToAttacks(observation['pos_args'], 
                                                                   observation['neg_args'],
                                                                  possible_graphs[chosen_graph])

        observation['rating'] = 10

        observations.append(observation)
        

    return observations

In [26]:
def runSingleSimulation(always_true, noObservations, appearance):

    graph_probabilities = []
    for i in range(5):
        observations = generateSyntheticObservations(noObservations, all_p_g, all_prior_distribs, appearance, always_true)
        target_distribution = {}
        target_distribution['pos_args'] = ['a','b','d']
        target_distribution['neg_args'] = ['c','e']
        target_distribution['rating'] = 10

        p_G = prior(target_distribution['pos_args'], target_distribution['neg_args'])

        # generate the prior distribution
        target_prior_distribution = p_G.getDistribution(target_distribution['rating'])

        p_G.rating = 10
        all_updates_distribution = []

        current_prior = target_prior_distribution
        
        for synthetic_observation in observations:
            p_G_T = liklihood(p_G, synthetic_observation)
            print(p_G_T.observation_type)
            s = time.time()
            liklihood_distribution = p_G_T.buildLiklihoodDistribution()
            print(time.time() - s)
            
            s = time.time()
            p_T_G = posterior(current_prior, liklihood_distribution)
            posterior_distribution = p_T_G.buildPosteriorDistribution()
            all_updates_distribution.append(posterior_distribution)
            print(time.time() - s)
            current_prior = posterior_distribution
        print('---')

        graph_probabilities.append(current_prior[0])

    return graph_probabilities

In [27]:
graph_probabilities_dict = {}

In [28]:
start = time.time()
always_true = [[('a', 'c')], [('b', 'c')], [('d', 'c')],[('a','e')],[('b','e')],[('d','e')]]
noObservations = 25
appearance = 0.25
graph_probabilities = runSingleSimulation(always_true, noObservations, appearance)
print(time.time() - start)

similar
1.562695026397705
0.00034999847412109375
matching
0.001505136489868164
2.4080276489257812e-05
similar
1.4811770915985107
3.409385681152344e-05
similar
1.6003739833831787
0.00026106834411621094
similar
1.5561189651489258
3.0994415283203125e-05
similar
1.560595989227295
3.409385681152344e-05
similar
1.5355119705200195
4.76837158203125e-05
matching
0.002849102020263672
3.695487976074219e-05
similar
1.644423246383667
3.0994415283203125e-05
similar
1.6745188236236572
2.9802322387695312e-05
matching
0.0004448890686035156
3.504753112792969e-05
similar
1.539801836013794
3.1948089599609375e-05
matching
0.003144979476928711
0.0001430511474609375
matching
0.001977205276489258
3.600120544433594e-05
similar
1.644266128540039
0.00020599365234375
similar
1.5417168140411377
3.314018249511719e-05
similar
1.6169359683990479
3.3855438232421875e-05
similar
1.502234935760498
0.00028705596923828125
similar
1.60280442237854
3.504753112792969e-05
similar
1.5458953380584717
3.3855438232421875e-05
simil

KeyboardInterrupt: 

In [ ]:
graph_probabilities_dict[float(appearance*10)] = graph_probabilities

In [ ]:
h = sorted(graph_probabilities_dict[float(appearance*10)])  #sorted
fit = stats.norm.pdf(h, np.mean(h), np.std(h))  #this is a fitting indeed

plt.plot(h,fit)
      #use this to draw histogram of your data

    
plt.show()

In [ ]:
fig = plt.figure()

plt.xlabel('P(G|T)')

for key, item in graph_probabilities_dict.items():
    if key == 0:
        continue 

    h = sorted(item)

    fit = stats.norm.pdf(h, np.mean(h), np.std(h))  #this is a fitting indeed

    noise_level = round(1 - key/10, 2)
    
    plt.plot(h,fit, label='noise='+ str(noise_level))
      #use this to draw histogram of your data
    plt.legend()
plt.show()

Save last figure

In [ ]:
# fig.savefig('figures/ultimatesimulation_noise'+ str(noise_level) +'_noobservations' + str(noObservations) +'_150trials.jpeg')

# Save agreeable graphs

In [ ]:
finalGraph = {
    'target_distribution': target_distribution,
    'always_true': always_true,
    'noObservations': noObservations,
    'appearance': appearance,
    'graph_probabilities_dict': graph_probabilities_dict
}

In [ ]:
finalGraphs.append(finalGraph)

In [ ]:
finalGraphsDF = pd.DataFrame(finalGraphs)

In [ ]:
finalGraphsDF.to_json('final_distributions.json')

In [ ]:
finalGraphsDF

In [ ]:
# Work through means of graphs

In [ ]:
def runTrialForGraph():
    always_true = [[('c','a')],[('c', 'b')]]
    
    for noObservations in [5,10,15,20,25]:
        graph_probabilities_dict = {}

        
        for appearance in [0.25,0.5,0.75]:
            
            graph_probabilities = runSingleSimulation(always_true, noObservations, appearance)
            graph_probabilities_dict[float(appearance*10)] = graph_probabilities
     
        finalGraph = {
            'target_distribution': target_distribution,
            'always_true': always_true,
            'noObservations': noObservations,
            'graph_probabilities_dict': graph_probabilities_dict
        }
        
        finalGraphs.append(finalGraph)
        
        print('completed ', noObservations, 'with ', always_true)

In [ ]:
runTrialForGraph()

In [ ]:
finalGraphsDF = pd.DataFrame(finalGraphs)
finalGraphsDF

In [ ]:
def convertIntoMeanGraph(always_true, appearance):
    finalGraphsDF

In [ ]:
for i in finalGraphsDF.iloc[20:25].itertuples():
    print(np.mean(i.graph_probabilities_dict[7.5]))
    

In [ ]:
three_arg_sampled_graphs = [(0,5,0),
                           (5,10,1598),
                           (10,15,2808),
                           (15,20,3083),
                           (20,25,4095)]

In [ ]:
all_mean_probs = []
# Compute the probabilitiy means

noiselevel = 5.0

for sampled_graph in three_arg_sampled_graphs:
    
    mean_probs = []
    for i in finalGraphsDF.iloc[sampled_graph[0]:sampled_graph[1]].itertuples():
        mean_probs.append(np.mean(i.graph_probabilities_dict[noiselevel]))
        
    all_mean_probs.append(mean_probs)
    
# Put the prior at the top of each list for each graph for plotting purposes

for idx, mean_prob in enumerate(all_mean_probs):
    mean_prob.insert(0, p_G_3args_prior_distribution[three_arg_sampled_graphs[idx][2]])

In [ ]:
fig = plt.figure()

for amp in all_mean_probs:
    
    plt.plot([0,5,10,15,20,25],amp)

plt.ylabel('P(G|T)')
plt.xlabel('No. of Observations')
plt.show()

In [ ]:
fig.savefig('figures/mean_simulations_3args_noise' + str(noiselevel/10) + '.png')

# Plot the average of the points as a single line with a band around it

In [ ]:
from scipy.stats import sem, t
from scipy import mean
confidence = 0.95

smooth_path = []
overline = []
underline = []
path_deviation = []
for all_mean in np.array(all_mean_probs).T:
    n = len(all_mean)
    m = mean(all_mean)
    std_err = sem(all_mean)
    h = std_err * t.ppf((1 + confidence) / 2, n - 1)
    
    smooth_path.append(m)
    path_deviation.append(h)
    overline.append(m+h)
    underline.append(m-h)

path_deviation = list(range(0,len(path_deviation)))

In [ ]:
fig = plt.figure()
plt.plot([0,5,10,15,20,25],smooth_path) #mean curve.
plt.fill_between([0,5,10,15,20,25], underline, overline, color='b', alpha=.1)
plt.ylabel('P(G|T)')
plt.xlabel('No. of Observations')
plt.show()
fig.savefig('figures/confidence_interval_3arg_noise_'+str(noiselevel/10)+'.png')

In [ ]:
#some example data
x= np.linspace(0.1, 9.9, 20)
y = 3.0 * x
#some confidence interval
ci = 1.96 * np.std(y)/np.mean(y)

fig, ax = plt.subplots()
ax.plot(x,y)
ax.fill_between(x, (y-ci), (y+ci), color='b', alpha=.1)
plt.show()

In [ ]:
(y+ci).shape

In [ ]:
plt.plot([5,10,15,20,25],smooth_path) #mean curve.
plt.fill_between([5,10,15,20,25], underline, overline, color='b', alpha=.1)
plt.show()

In [ ]:
np.array(overline).shape